In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time
import copy
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time

In [2]:
rootPath = 'd:/mt_data/1212_withrw/'
encodePath = rootPath+'encoded/t1/'

In [5]:
devices = [ 'pi3', 'pi4_2G', 'pi4_4G']
tws = [60]

In [8]:
encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format('pi4_2G', '60'))

In [5]:
cols = encoded_trace_df.columns

In [6]:
cols

Index(['ids', 'maltype', 'system calls frequency_1gram',
       'system calls tfidf_1gram', 'system calls hashing_1gram',
       'system calls frequency_2gram', 'system calls tfidf_2gram',
       'system calls hashing_2gram', 'system calls frequency_3gram',
       'system calls tfidf_3gram', 'system calls hashing_3gram',
       'system calls frequency_4gram', 'system calls tfidf_4gram',
       'system calls hashing_4gram', 'system calls frequency_5gram',
       'system calls tfidf_5gram', 'system calls hashing_5gram'],
      dtype='object')

In [7]:
ft = [x.tolist() for x in encoded_trace_df[cols[5]]]

In [8]:
df = pd.DataFrame([encoded_trace_df['ids'].tolist(), encoded_trace_df['maltype'].tolist(), ft]).transpose()

In [11]:
df.columns = ['ids', 'maltype', cols[2]]

In [12]:
df

,ids,maltype,system calls frequency_1gram
0,delay_0_2021-11-23,delay,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28809, 0, 0,..."
1,delay_100_2021-11-23,delay,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25768, 0, 0,..."
2,delay_101_2021-11-23,delay,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25094, 0, 0,..."
3,delay_102_2021-11-23,delay,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25377, 0, 0,..."
4,delay_103_2021-11-23,delay,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23190, 0, 0,..."
...,...,...,...
3595,spoof_96_2021-11-25,spoof,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26653, 0, 0,..."
3596,spoof_97_2021-11-25,spoof,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23787, 0, 0,..."
3597,spoof_98_2021-11-25,spoof,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25347, 0, 0,..."
3598,spoof_99_2021-11-25,spoof,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24196, 0, 0,..."


In [13]:
df = encoded_trace_df[['ids', 'maltype', cols[2]]]

In [3]:
def split(device, tw):
    print('start {} {}'.format(device, tw))
    encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format(device, tw))
    print('read {} {} finished'.format(device, tw))
    cols = encoded_trace_df.columns
    for i in range(2, len(cols)):
        if type(encoded_trace_df[cols[i]][0])!=list:
            ft = [x.tolist() for x in encoded_trace_df[cols[i]]]
        else:
            ft = [x for x in encoded_trace_df[cols[i]]]
        df = pd.DataFrame([encoded_trace_df['ids'].tolist(), encoded_trace_df['maltype'].tolist(), ft]).transpose()
        df.columns = ['ids', 'maltype', cols[i]]
        print(df.columns)
        df.to_csv(encodePath+'encoded_bow{}_{}_{}.csv'.format(device, tw, cols[i]), sep='\t', index=None)

In [6]:
for device in devices:
    for tw in [60]:
        split(device, tw)

start pi3 60
read pi3 60 finished
Index(['ids', 'maltype', 'system calls frequency_1gram'], dtype='object')
Index(['ids', 'maltype', 'system calls tfidf_1gram'], dtype='object')
Index(['ids', 'maltype', 'system calls hashing_1gram'], dtype='object')
Index(['ids', 'maltype', 'system calls frequency_2gram'], dtype='object')
Index(['ids', 'maltype', 'system calls tfidf_2gram'], dtype='object')
Index(['ids', 'maltype', 'system calls hashing_2gram'], dtype='object')
Index(['ids', 'maltype', 'system calls frequency_3gram'], dtype='object')
Index(['ids', 'maltype', 'system calls tfidf_3gram'], dtype='object')
Index(['ids', 'maltype', 'system calls hashing_3gram'], dtype='object')
Index(['ids', 'maltype', 'system calls frequency_4gram'], dtype='object')
Index(['ids', 'maltype', 'system calls tfidf_4gram'], dtype='object')
Index(['ids', 'maltype', 'system calls hashing_4gram'], dtype='object')
Index(['ids', 'maltype', 'system calls frequency_5gram'], dtype='object')
Index(['ids', 'maltype', 'sy